In [ ]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


LSTM's in Pytorch
~~~~~~~~~~~~~~~~~

Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [13]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(out.argmax(dim=2))
print(hidden)

tensor([[[-0.0667, -0.2373, -0.1526]],

        [[-0.1252, -0.2964, -0.2837]],

        [[-0.1207, -0.5949, -0.2282]],

        [[-0.1977, -0.2964, -0.1576]],

        [[-0.0398, -0.3752, -0.2361]]], grad_fn=<StackBackward>)
tensor([[0],
        [0],
        [0],
        [2],
        [0]])
(tensor([[[-0.0398, -0.3752, -0.2361]]], grad_fn=<StackBackward>), tensor([[[-0.1104, -0.9429, -0.4783]]], grad_fn=<StackBackward>))


Example: An LSTM for Part-of-Speech Tagging
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Create the model:



In [14]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        print('s = ',sentence)
        embeds = self.word_embeddings(sentence)
        print('e = ',embeds)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [15]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(torch.argmax(tag_scores,dim=1))

s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5472,  1.1474, -0.6696, -0.7714, -0.2665,  0.0449],
        [-0.7013, -2.8400,  0.4662,  0.0490, -0.3228, -0.5578],
        [ 0.8521,  2.3545, -0.8743, -0.4507, -1.0645,  1.3067],
        [-0.8167, -1.0331, -0.3161, -0.6191, -1.0883, -1.3507],
        [ 0.0410,  1.0469,  0.1030,  1.8462,  0.4964,  0.4062]])
tensor([[-0.8633, -1.5639, -0.9972],
        [-0.9625, -1.6224, -0.8659],
        [-0.8963, -1.5588, -0.9635],
        [-0.9364, -1.6388, -0.8825],
        [-0.8881, -1.6067, -0.9468]])
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5472,  1.1474, -0.6696, -0.7714, -0.2665,  0.0449],
        [-0.7013, -2.8400,  0.4662,  0.0490, -0.3228, -0.5578],
        [ 0.8521,  2.3545, -0.8743, -0.4507, -1.0645,  1.3067],
        [-0.8167, -1.0331, -0.3161, -0.6191, -1.0883, -1.3507],
        [ 0.0410,  1.0469,  0.1030,  1.8462,  0.4964,  0.4062]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.0258e+00,  1.2119e+00, -1

e =  tensor([[ 0.5515,  1.1503, -0.6655, -0.7682, -0.2597,  0.0428],
        [-0.7031, -2.8447,  0.4627,  0.0550, -0.3233, -0.5541],
        [ 0.8577,  2.3470, -0.8798, -0.4459, -1.0657,  1.3067],
        [-0.8211, -1.0297, -0.3125, -0.6144, -1.0866, -1.3538],
        [ 0.0390,  1.0464,  0.0958,  1.8470,  0.5036,  0.4126]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.0293,  1.2148, -0.1602,  0.2283, -0.0035,  0.0207],
        [ 1.1546,  0.2264, -1.7690, -0.4106, -0.7658,  0.5555],
        [-0.0801, -2.3883, -0.5824,  0.9442,  0.2729, -0.4345],
        [-1.0830,  2.4131, -0.5615, -2.6282,  0.2140,  0.0151]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5518,  1.1503, -0.6651, -0.7680, -0.2596,  0.0424],
        [-0.7032, -2.8450,  0.4625,  0.0551, -0.3233, -0.5539],
        [ 0.8581,  2.3465, -0.8802, -0.4455, -1.0659,  1.3068],
        [-0.8213, -1.0298, -0.3123, -0.6141, -1.0863, -1.3541],
        [ 0.0388,  1.046

e =  tensor([[ 0.5570,  1.1451, -0.6515, -0.7615, -0.2628,  0.0275],
        [-0.7057, -2.8491,  0.4612,  0.0554, -0.3238, -0.5503],
        [ 0.8735,  2.3305, -0.8970, -0.4315, -1.0753,  1.3117],
        [-0.8255, -1.0395, -0.3082, -0.6053, -1.0761, -1.3636],
        [ 0.0339,  1.0390,  0.0917,  1.8516,  0.5180,  0.4168]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.0355,  1.2254, -0.1629,  0.2108, -0.0093,  0.0360],
        [ 1.1659,  0.2355, -1.7962, -0.4315, -0.7937,  0.5710],
        [-0.0832, -2.4000, -0.5904,  0.9561,  0.2820, -0.4387],
        [-1.0851,  2.4180, -0.5622, -2.6449,  0.2207,  0.0146]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5572,  1.1447, -0.6508, -0.7613, -0.2632,  0.0266],
        [-0.7058, -2.8491,  0.4612,  0.0553, -0.3238, -0.5502],
        [ 0.8746,  2.3296, -0.8981, -0.4307, -1.0760,  1.3121],
        [-0.8257, -1.0403, -0.3080, -0.6048, -1.0755, -1.3642],
        [ 0.0336,  1.038

e =  tensor([[-1.0549e+00,  1.2475e+00, -1.6112e-01,  1.8127e-01, -2.2761e-03,
          5.8743e-02],
        [ 1.1994e+00,  2.6043e-01, -1.8700e+00, -4.6351e-01, -8.2998e-01,
          6.0978e-01],
        [-7.7368e-02, -2.4233e+00, -6.1233e-01,  9.6698e-01,  2.8735e-01,
         -4.3576e-01],
        [-1.0979e+00,  2.4298e+00, -5.5908e-01, -2.6758e+00,  2.3939e-01,
          1.1418e-02]], grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5637,  1.1325, -0.6257, -0.7605, -0.2832, -0.0097],
        [-0.7112, -2.8459,  0.4704,  0.0496, -0.3208, -0.5500],
        [ 0.9236,  2.2981, -0.9563, -0.4036, -1.1061,  1.3330],
        [-0.8321, -1.0758, -0.3035, -0.5884, -1.0514, -1.3846],
        [ 0.0186,  1.0157,  0.1002,  1.8718,  0.5580,  0.4129]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.0561e+00,  1.2484e+00, -1.6086e-01,  1.8003e-01, -1.4272e-03,
          5.9496e-02],
        [ 1.2009e+00,  2.6150e-01, -1.8730e+00, -4.6470e-0

s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.0969,  1.2762, -0.1546,  0.1398,  0.0318,  0.0756],
        [ 1.2425,  0.2955, -1.9563, -0.4988, -0.8657,  0.6530],
        [-0.0586, -2.4388, -0.6589,  0.9564,  0.2774, -0.4280],
        [-1.1200,  2.4436, -0.5514, -2.7056,  0.2632,  0.0046]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5631,  1.1125, -0.5873, -0.7748, -0.3065, -0.0699],
        [-0.7251, -2.8425,  0.5028,  0.0525, -0.3021, -0.5538],
        [ 0.9925,  2.2712, -1.0417, -0.3793, -1.1482,  1.3694],
        [-0.8336, -1.1275, -0.3129, -0.5705, -1.0275, -1.4074],
        [-0.0148,  0.9953,  0.1224,  1.8956,  0.6247,  0.4038]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.0985,  1.2774, -0.1546,  0.1382,  0.0331,  0.0760],
        [ 1.2441,  0.2969, -1.9594, -0.5000, -0.8669,  0.6545],
        [-0.0577, -2.4391, -0.6612,  0.9554,  0.2766, -0.4278],
        [-1.1207,  2.4441, -0.5511, -2.7065,  0.2639,  0.0043]

...

e =  tensor([[-1.1731,  1.3604, -0.2074,  0.0316,  0.0768,  0.1174],
        [ 1.3566,  0.3950, -2.1647, -0.5633, -0.9420,  0.7885],
        [ 0.1106, -2.4410, -0.9726,  0.8704,  0.1240, -0.4426],
        [-1.1721,  2.4654, -0.5289, -2.7382,  0.3124, -0.0080]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.4653,  0.9575, -0.3793, -0.7881, -0.2719, -0.4770],
        [-0.9846, -2.7667,  0.7310,  0.0505, -0.0275, -0.5541],
        [ 1.1397,  2.2319, -1.2447, -0.3000, -1.2415,  1.4904],
        [-0.7545, -1.2452, -0.4670, -0.5159, -1.0549, -1.4320],
        [-0.1148,  1.0402,  0.1047,  1.8434,  0.7783,  0.3739]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.1736,  1.3609, -0.2079,  0.0310,  0.0771,  0.1180],
        [ 1.3574,  0.3956, -2.1662, -0.5636, -0.9426,  0.7900],
        [ 0.1122, -2.4413, -0.9748,  0.8704,  0.1227, -0.4427],
        [-1.1726,  2.4654, -0.5286, -2.7383,  0.3128, -0.0081]],
       grad_fn=<Embeddi

s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.1892,  1.3740, -0.2195,  0.0171,  0.0843,  0.1327],
        [ 1.3793,  0.4100, -2.2022, -0.5705, -0.9581,  0.8299],
        [ 0.1441, -2.4487, -1.0223,  0.8735,  0.0949, -0.4438],
        [-1.1851,  2.4668, -0.5219, -2.7411,  0.3241, -0.0095]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.4750,  0.8851, -0.3836, -0.7545, -0.2625, -0.6219],
        [-1.0360, -2.7486,  0.7428,  0.0207,  0.0190, -0.5707],
        [ 1.1682,  2.2264, -1.2872, -0.2790, -1.2629,  1.5226],
        [-0.7232, -1.2665, -0.5148, -0.4979, -1.0743, -1.4273],
        [-0.1342,  1.0481,  0.0921,  1.8254,  0.7982,  0.3666]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.1897,  1.3743, -0.2198,  0.0168,  0.0845,  0.1331],
        [ 1.3799,  0.4104, -2.2033, -0.5707, -0.9586,  0.8312],
        [ 0.1449, -2.4489, -1.0235,  0.8736,  0.0942, -0.4438],
        [-1.1855,  2.4668, -0.5217, -2.7412,  0.3244, -0.0096]

s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 4.9795e-01,  8.3341e-01, -4.1524e-01, -7.2855e-01, -2.6109e-01,
         -7.1464e-01],
        [-1.0661e+00, -2.7407e+00,  7.4992e-01,  2.0596e-03,  4.4307e-02,
         -5.8358e-01],
        [ 1.1888e+00,  2.2229e+00, -1.3180e+00, -2.6360e-01, -1.2806e+00,
          1.5466e+00],
        [-6.9961e-01, -1.2814e+00, -5.5239e-01, -4.8257e-01, -1.0894e+00,
         -1.4200e+00],
        [-1.4896e-01,  1.0533e+00,  8.3053e-02,  1.8123e+00,  8.1148e-01,
          3.6083e-01]], grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.2034,  1.3818, -0.2277,  0.0090,  0.0915,  0.1431],
        [ 1.3961,  0.4206, -2.2298, -0.5750, -0.9712,  0.8645],
        [ 0.1630, -2.4547, -1.0522,  0.8788,  0.0777, -0.4431],
        [-1.1954,  2.4672, -0.5162, -2.7429,  0.3335, -0.0113]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 4.9875e-01,  8.3204e-01, -4.1642e-01, -7.2786e-01, -2.6113e-01,
         -7.1702e-0

s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5206,  0.7989, -0.4500, -0.7115, -0.2634, -0.7728],
        [-1.0870, -2.7364,  0.7550, -0.0104,  0.0609, -0.5930],
        [ 1.2039,  2.2204, -1.3403, -0.2523, -1.2945,  1.5640],
        [-0.6809, -1.2923, -0.5829, -0.4701, -1.1019, -1.4129],
        [-0.1606,  1.0572,  0.0764,  1.8023,  0.8211,  0.3562]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([5, 6, 7, 8])
e =  tensor([[-1.2147,  1.3857, -0.2331,  0.0047,  0.0979,  0.1484],
        [ 1.4076,  0.4274, -2.2490, -0.5776, -0.9806,  0.8892],
        [ 0.1747, -2.4591, -1.0719,  0.8837,  0.0663, -0.4422],
        [-1.2028,  2.4670, -0.5120, -2.7441,  0.3406, -0.0134]],
       grad_fn=<EmbeddingBackward>)
s =  tensor([0, 1, 2, 3, 4])
e =  tensor([[ 0.5213,  0.7979, -0.4512, -0.7110, -0.2635, -0.7744],
        [-1.0876, -2.7363,  0.7551, -0.0107,  0.0613, -0.5933],
        [ 1.2043,  2.2203, -1.3410, -0.2519, -1.2949,  1.5645],
        [-0.6804, -1.2926, -0.5839, -0.4697, -1.1023, -1.41

tensor([0, 1, 2, 0, 1])


Exercise: Augmenting the LSTM part-of-speech tagger with character-level features
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.




In [ ]:
class LSTMCharTagger(nn.Module):

    def __init__(self, 
                 word_embedding_dim, 
                 char_embedding_dim,
                 word_hidden_dim,
                 char_hidden_dim,
                 vocab_size,
                 char_size,
                 char_target_size,
                 word_target_size):
        super(LSTMCharTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.char_embeddings = nn.Embedding(char_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim,char_hidden_dim)
        self.char_h2t = nn.Linear(chard_hidden_dim,char_target_size)

        self.word_embeddings = nn.Embedding(vocab_size, word_embedding_dim)
        self.word_lstm = nn.LSTM(embedding_dim, word_hidden_dim)
        self.word_h2t = nn.Linear(word_hidden_dim, word_target_size)

    def forward(self, sentence):
        char_embeds = self.char_embeddings(char)
        word_embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores